## S&P 500 Index Changes Data Processing

This notebook processes HTML data from Wikipedia to extract S&P 500 component changes with announcement dates.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

# Create data folder if it doesn't exist
os.makedirs('data', exist_ok=True)

### Step 1: Parse Component Changes from HTML

In [ ]:
# Read the HTML file
with open('./data/ComponentUpdate.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'changes'})

# Extract data from table
data = []
rows = table.find('tbody').find_all('tr')

for row in rows:
    cells = row.find_all('td')
    if len(cells) >= 5:
        date = cells[0].get_text(strip=True)
        added_ticker = cells[1].get_text(strip=True)
        added_security = cells[2].get_text(strip=True)
        removed_ticker = cells[3].get_text(strip=True)
        removed_security = cells[4].get_text(strip=True)
        reason_cell = cells[5] if len(cells) > 5 else None
        
        # Extract reference numbers
        reference_numbers = []
        if reason_cell:
            sup_tags = reason_cell.find_all('sup', class_='reference')
            for sup in sup_tags:
                cite_id = sup.get('id', '')
                if cite_id:
                    match = re.search(r'cite_ref-(?:[^_]+_)?(\d+)', cite_id)
                    if match:
                        reference_numbers.append(match.group(1))
        
        # Get clean reason text
        reason = cells[5].get_text(strip=True) if len(cells) > 5 else ''
        reason = re.sub(r'\[\d+\]', '', reason)
        
        data.append({
            'Effective_Date': date,
            'Added_Ticker': added_ticker,
            'Added_Security': added_security,
            'Removed_Ticker': removed_ticker,
            'Removed_Security': removed_security,
            'Reason': reason.strip(),
            'Reference_Numbers': ','.join(reference_numbers) if reference_numbers else ''
        })

# Create DataFrame
df = pd.DataFrame(data)

# Convert Date column to datetime
df['Effective_Date'] = pd.to_datetime(df['Effective_Date'], format='%B %d, %Y', errors='coerce')

print(f"Extracted {len(df)} component changes")

Extracted 375 component changes


### Step 2: Extract Announcement Dates from References

In [ ]:
# Read the Reference.html file
with open('./data/Reference.html', 'r', encoding='utf-8') as file:
    ref_html_content = file.read()

ref_soup = BeautifulSoup(ref_html_content, 'html.parser')

def extract_all_dates_from_text(text):
    """Extract all dates from text using multiple patterns."""
    dates_found = []
    
    # Clean the text
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    
    # Pattern 1: YYYY-MM-DD format
    pattern1 = r'(\d{4}-\d{2}-\d{2})'
    for match in re.findall(pattern1, text):
        try:
            dates_found.append(datetime.strptime(match, '%Y-%m-%d'))
        except:
            pass
    
    # Pattern 2: Month DD, YYYY format
    pattern2 = r'((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},?\s+\d{4})'
    for match in re.findall(pattern2, text):
        try:
            clean_match = re.sub(r',', '', match)
            dates_found.append(datetime.strptime(clean_match, '%B %d %Y'))
        except:
            pass
    
    # Pattern 3: YYYYMMDD in URLs
    pattern3 = r'/(\d{8})-'
    for match in re.findall(pattern3, text):
        try:
            dates_found.append(datetime.strptime(match, '%Y%m%d'))
        except:
            pass
    
    return dates_found

# Extract references with their dates
reference_dates = {}
all_references = ref_soup.find_all('li', id=re.compile(r'cite_note-'))

for ref_item in all_references:
    ref_id = ref_item.get('id', '')
    ref_num_match = re.search(r'cite_note-(?:.*?-)?(\d+)$', ref_id)
    
    if ref_num_match:
        ref_num = ref_num_match.group(1)
        ref_text = ref_item.get_text()
        dates = extract_all_dates_from_text(ref_text)
        
        if dates:
            # Get the earliest date (most relevant for trading)
            reference_dates[ref_num] = min(dates)

print(f"Extracted dates from {len(reference_dates)} references")

Extracted dates from 266 references


### Step 3: Merge Announcement Dates and Create Final Dataset

In [4]:
# Add announcement dates to the main dataframe
df['Announcement_Date'] = pd.NaT

for idx, row in df.iterrows():
    ref_nums = str(row['Reference_Numbers']).split(',')
    ref_nums = [num.strip() for num in ref_nums if num.strip()]
    
    if ref_nums:
        dates = []
        for ref_num in ref_nums:
            if ref_num in reference_dates:
                dates.append(reference_dates[ref_num])
        
        if dates:
            df.at[idx, 'Announcement_Date'] = min(dates)

# Select only required columns in specified order
final_df = df[['Announcement_Date', 'Effective_Date', 'Added_Ticker', 'Added_Security', 
               'Removed_Ticker', 'Removed_Security', 'Reason']]

# Sort by Effective_Date (most recent first)
final_df = final_df.sort_values('Effective_Date', ascending=False)

# Save to data folder
output_path = 'data/sp500_index_changes.csv'
final_df.to_csv(output_path, index=False)

print(f"\n✅ Dataset saved to {output_path}")
print(f"Total records: {len(final_df)}")
print(f"Records with announcement dates: {final_df['Announcement_Date'].notna().sum()}")

# Display sample
print("\nFirst 5 records:")
print(final_df.head())


✅ Dataset saved to data/sp500_index_changes.csv
Total records: 375
Records with announcement dates: 342

First 5 records:
  Announcement_Date Effective_Date Added_Ticker    Added_Security  \
0        2025-07-18     2025-07-23          XYZ       Block, Inc.   
1        2025-07-14     2025-07-18          TTD  Trade Desk (The)   
2        2025-07-02     2025-07-09         DDOG           Datadog   
3        2025-05-12     2025-05-19         COIN          Coinbase   
4        2025-03-07     2025-03-24         DASH          DoorDash   

  Removed_Ticker    Removed_Security  \
0            HES    Hess Corporation   
1           ANSS               Ansys   
2           JNPR    Juniper Networks   
3            DFS  Discover Financial   
4            BWA          BorgWarner   

                                              Reason  
0  S&P 500 and S&P 100 constituent Chevron Corp. ...  
1  S&P 500 constituent Synopsys Inc. acquired Ansys.  
2  S&P 500 constituent Hewlett Packard Enterprise...  
3